In [1]:
# Numeric
import numpy as np
import pandas as pd


# Tools
import os, sys
import re
import string
import csv
import itertools

from pattern.it import parse
from pattern.it import pprint

from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords as sw
from nltk.stem.snowball import SnowballStemmer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler

from sklearn.decomposition import TruncatedSVD


# Clustering
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering



# Classifiers
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import make_pipeline



# Metrics
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score


# Visualization
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
import spacy
nlp = spacy.load("it_core_news_sm")
from spacy import lemmatizer
stemmer = PorterStemmer()

# tokens = nlp(dataset.iloc[0,0])

# for token in tokens:
#     print(token.text, token.lemma_, stemmer.stem(token.lemma_))



In [3]:
# print(nlp.pipe_names)

In [4]:
# lemmas = set()

# for doc in nlp.pipe(dataset['text'], disable=["tagger", "ner", "parser", "tagger", "textcat"]):
#     for token in doc:
#         lemmas.add(token.lemma_)


In [5]:
# lemmas

In [6]:
# lemmas = []
# for index, document in dataset.iterrows():
#     tokens = nlp(row['text'])
#     for doc in nlp.pipe(texts, disable=["tagger", "parser"]):
#     for token in tokens:
#         if token.text not in string.punctuation and len(token.text) > 3 and len(token.text) < 16:
#             print(token.text)
#             lemmas.append(token.text)
    

In [7]:
# lemmas

In [8]:
directory = "/Users/lpdef/Desktop/Polito/Data Science lab process and methods/Final Exam/dataset_winter_2020/"
dataset = pd.read_csv(directory + "development.csv")
testset = pd.read_csv(directory + "evaluation.csv")

corpus = pd.concat([dataset.iloc[:,0],testset.iloc[:,0]])

dataset.head()
dataset.iloc[:,0].size

28754

In [9]:
X_train = dataset.iloc[:,0]
# doc = nlp(dataset.iloc[0,0])
# for token in doc:
#     print(token.text, token.lemma_, token.pos_, token.tag_)

In [14]:

spacy_stopWords = spacy.lang.it.stop_words.STOP_WORDS

class LemmaTokenizer(object):
    
    def __init__(self):
        self.lemmatizer = spacy.load("it_core_news_sm", disable=["ner", "parser", "tagger", "textcat"])
        self.Porter_Stemmer = SnowballStemmer("italian")
        
        
    def __call__(self, document):
#         print(document)
        lemmas = []
        re_digit = re.compile("[0-9]")
        document = document.replace("'", " ")

        for t in self.lemmatizer(document):
            if bool(re.match("[0-9]+", t.lemma_))== False:
                t = t.lemma_.strip()
                t = t.translate(str.maketrans('', '', string.punctuation))
                
                if bool(t.isalpha())==True:
                    if len(t) > 1 and len(t) < 20:
                        t = self.Porter_Stemmer.stem(t)
                        lemmas.append(t)

        
            
        return lemmas
    

stopWords = sw.words('italian')
stopWords.remove('ma')
stopWords.remove('con')
stopWords.extend(['avra', 'avro', 'sar', 'aver', 'com', 'contr', 'dar', 'esser', 'far', 'fec', 'foss', 
                  'lor', 'nostr', 'perc', 'qual', 'quant', 'quell', 'quest', 'star', 'stemm', 
                  'stess', 'stetter', 'tutt', 'vostr',
                 
                 'dio', 'neo', 'piu',
                  
                 'luned', 'marted', 'mercoled', 'gioved', 'venerd', 'sabat', 'domenic',
                  
                 'aglo', 'avere', 'avutare', 'avutere', 'dallare', 'ebbo', 'essere',
                 'facciata', 'facere', 'fara', 'faro', 'farsene', 'fossa', 'fosso', 'mieo', 'perca',
                 'sara', 'saro', 'stara', 'stare', 'stessa', 'staro', 'stesso', 'suoo', 'tuoo', 'aver',
                 'avut', 'fac', 'facc', 'farse', 'nostr'
                 
                 'fars', 'farsen'])
                 
#                  'gennai', 'febbrai', 'marz', 'april', 'magg', 'giungn', 'lugl', 'agost', 'settembr', 'ottobr', 'novembr', 'dicembr',
#                  'lorenz', 'anna', 'marc', 'francesc',

#                 'molt, '])


# stopWords.extend(spacy_stopWords)

lemmaTokenizer = LemmaTokenizer()
vectorizer = TfidfVectorizer(tokenizer=lemmaTokenizer, stop_words=stopWords, max_df=0.8, min_df=0.0005, 
                             strip_accents='unicode', lowercase=True, ngram_range=(1,3), sublinear_tf=True)


# X_train = vectorizer.fit_transform(X_train)

In [15]:
# lista= [dataset.iloc[2300,0],]
# vectorizer.fit(np.array(lista))
vectorizer.fit(X_train)

/Users/lpdef/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['fars'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.8, max_features=None,
                min_df=0.0005, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['ad', 'al', 'allo', 'ai', 'agli', 'all', 'agl',
                            'alla', 'alle', 'col', 'coi', 'da', 'dal', 'dallo',
                            'dai', 'dagli', 'dall', 'dagl', 'dalla', 'dalle',
                            'di', 'del', 'dello', 'dei', 'degli', 'dell',
                            'degl', 'della', 'delle', 'in', ...],
                strip_accents='unicode', sublinear_tf=True,
                token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<__main__.LemmaTokenizer object at 0x1a3b4d3910>,
                use_idf=True, vocabulary=None)

In [26]:
X_train = vectorizer.transform(X_train)

In [27]:
Y_test = vectorizer.transform(testset.iloc[:,0])

In [ ]:
# print("mal-" in vectorizer.get_feature_names())
# print("pess-" in vectorizer.get_feature_names())
# print("malissim" in vectorizer.get_feature_names())
# print("problem-" in vectorizer.get_feature_names())
# print("buon-" in vectorizer.get_feature_names())
# print("sconsigl-" in vectorizer.get_feature_names())

In [16]:
len(vectorizer.get_feature_names())

21496

### Clustering

In [ ]:
# from sklearn.neighbors import NearestNeighbors

# # Given matrix X and min_npoints, plot K-Neighbors chart
# # It also plots a horizontal line in correspondence to eps value
# def plot_neighbors(X, min_points, eps=0):
#     # "min_points + 1" because se first column in "distances" is the distance of each point and itself
#     nbrs = NearestNeighbors(n_neighbors=min_points + 1, n_jobs = -1).fit(X)
#     distances, indices = nbrs.kneighbors(X)

#     k_dist = distances[:, -1]
#     k_dist.sort()
#     x_axis = np.arange(k_dist.shape[0])

#     ### Disable comment below to activate interactive plots
#     #%matplotlib notebook

#     fig, ax = plt.subplots()
#     ax.plot(x_axis, k_dist, linestyle='', marker='o', markersize=1)
    
#     # Plot a horizontal line in correspondence to eps value
#     ax.hlines(eps, x_axis.min(), x_axis.max(), linestyle='--')
#     ax.set_title(f"K-Neighbors chart. Min_points = {min_points}, eps={eps}.")
#     plt.show()

In [ ]:
# # Plot charts
# plot_neighbors(X_train, 300, eps=0)

In [ ]:
# stopWords.remove('ma')
# # stopWords.remove('con')
# 'ma' in stopWords

In [ ]:
# # Run clustering
# model_db10 = DBSCAN(eps = 1.30, min_samples=300, n_jobs = -1)
# # model_db20 = DBSCAN(eps = 0.1, min_samples=20)
# res_db10 = model_db10.fit_predict(X_train)
# # res_db20 = model_db20.fit_predict(X_noise)

# # Show results
# # %matplotlib inline
# # fig, ax = plt.subplots(1, 2, figsize=(12, 5))
# # ax[0].set_title("DBSCAN, minpoints 10")
# # # ax[1].set_title("DBSCAN, minpoints 20")
# # ax[0].scatter(X_train[:,0], X_train[:,1], s=10, c=res_db10)
# # # ax[1].scatter(X_noise[:,0], X_noise[:,1], s=10, c=res_db20)
# # plt.show()

In [ ]:
# model_db10.labels_[model_db10.labels_==-1]

In [ ]:
# X_train.shape

### Fine clustering

In [ ]:

# l = set()

# for doc in nlp.pipe(vectorizer.get_feature_names(), disable=["tagger", "ner", "parser", "textcat"]):
#     for token in doc:
#         print(token.text, token.lemma_)


In [ ]:
# tokens = nlp("abbandonato")
# for token in tokens:
#     print(token.text, token.lemma_)
    
    
# for doc in nlp.pipe(["abbandonato",], disable=["tagger", "ner", "parser", "textcat"]):
#     for token in doc:
#         print(token.text, token.lemma_)

In [ ]:
# param_grid = {'C':list(range(2,10,2)), 'penalty':['l2'], 'solver':['liblinear']}
# y_train = dataset.iloc[:, 1]

# clf = LogisticRegression(verbose=1, random_state=0, max_iter=10000)

# gridSearch = GridSearchCV(clf, param_grid, scoring='f1_weighted', cv=5)

# gridSearch.fit(X_train, y_train)

In [ ]:
# print(gridSearch.best_params_)

In [ ]:
# method = TSNE(n_components=2, random_state=0)
# Y = method.fit_transform(red_X)

In [ ]:
# Y

In [ ]:
# LABEL_COLOR_MAP = {'pos' : 'r',
#                    'neg' : 'k'}
# label_color = [LABEL_COLOR_MAP[l] for l in y_train]

In [ ]:
# fig, ax = plt.subplots(figsize=(10, 10))
# ax.scatter(Y[:, 0], Y[:, 1], cmap=plt.cm.Spectral, c=label_color)

In [28]:
y_train = dataset.iloc[:, 1]

# clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
clf = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=6, penalty='l2',max_iter=1000)
# clf = SVC()
# clf = SVC(kernel='sigmoid', gamma=1.0, degree=15, random_state=0)
# clf = SVC(kernel='sigmoid', gamma=1.0)





# pipeline = make_pipeline(MaxAbsScaler(), clf)
# pipeline.fit(X_train, y_train)

f1_cv = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1_weighted')

print(f"Macro-f1 for each iteration: {f1_cv}")
mean_macro_f1 = f1_cv.mean()
std_macro_f1 = f1_cv.std() * 2
print(f"Macro-f1 (statistics): {mean_macro_f1:.3f} (+/- {std_macro_f1:.3f})")

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Macro-f1 for each iteration: [0.96653241 0.96979185 0.96001167 0.96212441 0.96348172]
Macro-f1 (statistics): 0.964 (+/- 0.007)


In [29]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_train)

print()
conf_mat = confusion_matrix(y_train, y_pred)
print(conf_mat)
count = 0;
for i, label in enumerate(y_pred):
    if y_train[i] is not label:
        count = count + 1
        print(i,y_train[i],label)

[LibLinear]
[[ 9101   121]
 [   63 19469]]
24 pos neg
42 neg pos
131 neg pos
337 pos neg
593 pos neg
700 neg pos
749 neg pos
806 neg pos
826 pos neg
1062 neg pos
1149 pos neg
1410 pos neg
1526 pos neg
1614 neg pos
1695 neg pos
1898 pos neg
2090 neg pos
2096 pos neg
2393 neg pos
2562 neg pos
2844 neg pos
2851 neg pos
2893 pos neg
3134 neg pos
3467 neg pos
3469 pos neg
3502 neg pos
3525 neg pos
3618 pos neg
3779 pos neg
3998 neg pos
4652 neg pos
4715 pos neg
4740 pos neg
5209 pos neg
5419 neg pos
5481 neg pos
5569 pos neg
5590 neg pos
5598 pos neg
6081 neg pos
6547 neg pos
6690 pos neg
6785 neg pos
6824 neg pos
6990 neg pos
7058 neg pos
7087 neg pos
7123 neg pos
7303 neg pos
7340 neg pos
7554 neg pos
8679 pos neg
9207 neg pos
9291 neg pos
9849 neg pos
10555 pos neg
10575 neg pos
10672 neg pos
10849 neg pos
10860 neg pos
10881 neg pos
10946 neg pos
11083 neg pos
11140 neg pos
11717 pos neg
11774 pos neg
11801 neg pos
12122 neg pos
12144 neg pos
12147 pos neg
12388 neg pos
12548 neg pos
12

In [32]:
count

184

In [33]:
dataset.iloc[1149,0]

"Tipo di tasso difficile.\nIl punto è: se si considera ciò che si paga e quello che si ottiene, si cerca un raffinato hotel a quattro stelle.\nSe vi aspettate un esperienza a cinque stelle al prezzo di un 4 stelle, allora resterete delusi.\n\nNon voglio entrare nei dettagli. Appena livello le vostre aspettative di un four-star facility e non ve ne pentirete avendo scelto Golden Palace.\nSolo una raccomandazione: dato che non c'è servizio di lavanderia all'interno dei locali (se ho azzeccato, usano un nelle vicinanze Independent business che, come ci si potrebbe aspettare, chiude a un certo punto nel pomeriggio e durante il fine settimana), e nemmeno le camere sono dotate di un ferro da stiro, è meglio avere una camicia stirata correttamente nel vostro bagaglio se arrivate all'hotel dall'aeroporto e sono state pianificando di andare direttamente a cena fuori dopo la doccia!"

In [34]:
clf.fit(X_train, y_train)

y_pred = clf.predict(Y_test)


[LibLinear]

In [35]:
testset.shape

(12323, 1)

In [36]:
def dump_to_file(filename, labels):
    """Dump the evaluated labels to a CSV file."""
    with open(filename, mode='w', encoding='UTF-8') as f:
        fwriter = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        fwriter.writerow(["Id", "Predicted"])
        for Id, label in enumerate(labels):
            fwriter.writerow([Id, label])
            

dump_to_file("Labels.csv", y_pred)

In [37]:
print(testset.iloc[1000,0])
print()
print(y_pred[1000])

y_pred.shape[0] == testset.shape[0]


Il nostro primo soggiorno a Napoli e abbiamo scelto questo hotel dal British Airways" consigliato elenco - e siamo contenti così. Ottima vista sulla Baia verso Capri e Castell dell'Ovo con molti servizi locali - soprattutto ristoranti, ma anche ottimi collegamenti bus route, una vicina fermata del taxi e un traffico - gratis a piedi la città lungo il fronte mare o sulla collina lungo la Via Chiaia.
L'hotel stesso offre un'ottima colazione e camere eleganti, ma consigliamo pagare di più per la vista sul mare.

pos


True

In [38]:
'stara' in vectorizer.get_feature_names()

# vectorizer.get_feature_names()

False

In [ ]:
vectorizer.get_feature_names()

In [ ]:
# tokens = nlp("andavo", disable=["tagger", "ner", "parser", "tagger", "textcat"])
# for token in tokens:
#     print(token.lemma_)
